In [1]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [2]:
def delevel_query(query_list):
    line_level = []
    sub_query = []
    pos_delete, pos_where = len(query_list), len(query_list)
    
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete = i
        if line.startswith('WHERE'):
            pos_where = i 
        if line.startswith('FROM'):
            pos_from = i-1
        if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
            pos_join = i
  
    for line in query_list[:pos_from+2]:
        line_level.append((line, "level_1"))
        
    for line in query_list[pos_from+1:pos_where]:
        if line.startswith(' ') or line.startswith('FROM'):
            sub_query.append(line[3:])
        else:
            line_level.append((line, "level_1"))

    for line in query_list[pos_where:pos_delete]:    
        line_level.append((line, "level_1"))

    return line_level, sub_query[1:]

In [3]:
def parse_sub_query(sub_query_list):
    sub_query = "\n".join(sub_query_list)
    sub_query = sub_query.lstrip('\n').lstrip(' ')
    
    formatter = column_parser.Parser(sub_query)
    formatted = formatter.format_query(sub_query)
    sub_query_list = formatted.split('\n')

    query = "\n".join(sub_query_list).lstrip('\n').lstrip(' ').lstrip('\(')
    query_list = query.split('\n')
    
    return query_list

In [4]:
def has_child(sub_query_list):
    query_list = parse_sub_query(sub_query_list)
    query_list = delevel_query(query_list)[1]
    return query_list, query_list != []

In [5]:
query_1 = """SELECT * FROM sfdc.accounts sfdc_accounts
LEFT JOIN (SELECT MAX(dt) FROM sfdc.cases) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03' 
"""


In [ ]:
query_2 = """SELECT *
FROM (
  SELECT u.name, b.customer_tier_c, b.name, m.account, b.x18_digit_account_id_c, s.id, m.platform, m.mobile_os, m.num_requests, Row_number() OVER(
    PARTITION BY s.id
  ) row_
  FROM wbr.map_requests_by_account m
  INNER JOIN (
    SELECT DISTINCT id
    FROM mapbox_customer_data.styles 
    WHERE cast(dt as DATE) >= CURRENT_DATE - INTERVAL '14' DAY
      AND sources LIKE '%mapbox-streets-v7%'
  ) s
  ON m.service_metadata_version = s.id
  LEFT JOIN (
    SELECT customer_tier_c, csm_c, name, mapbox_username_c, x18_digit_account_id_c
    FROM sfdc.accounts
    WHERE cast(dt as DATE) = CURRENT_DATE - INTERVAL '1' DAY
  ) b
  ON m.account = b.mapbox_username_c
  LEFT JOIN (
    SELECT name, id
    FROM sfdc.users
    WHERE cast(dt as DATE) = CURRENT_DATE - INTERVAL '1' DAY
  ) u
  ON b.csm_c = u.id
  WHERE cast(m.dt as DATE) >= CURRENT_DATE - INTERVAL '14' DAY
    AND m.service_metadata = 'custom'
    AND m.service = 'styles'
    AND b.customer_tier_c IN (
      'Tier 0',
      'Tier 1',
      'Tier 2',
      'Tier 3',
      'Tier 4'
    )
)
WHERE row_ = 1
AND m.service_metadata = 'custom'
ORDER BY 1, 4
LIMIT 5000
"""

In [7]:
formatter = column_parser.Parser(query_1)
formatted = formatter.format_query(query_1)
query_list = formatted.split('\n')
query_list

['SELECT *',
 'FROM sfdc.accounts sfdc_accounts',
 'LEFT JOIN',
 '  (SELECT MAX(dt)',
 '   FROM sfdc.cases) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt',
 "WHERE dt > '2020-04-03'"]

In [8]:
formatter = column_parser.Parser(query_1)
formatted = formatter.format_query(query_1)
query_list = formatted.split('\n')
sub_query_list, has_subquery = has_child(query_list)

In [9]:
sub_query_list

['SELECT MAX(dt)',
 'FROM sfdc.cases) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt']

In [10]:
has_subquery

True

In [11]:
sub_sub_query_list, has_subquery = has_child(sub_query_list)

In [12]:
sub_sub_query_list

[]

In [13]:
has_subquery

False